In [2]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sqlalchemy-2.0.43-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp311-cp311-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp311-cp311-win_amd64.whl.metadata (76 kB)
  Using cac


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

loader = Docx2txtLoader('./data/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)
print(len(document_list))
# print(f'각 청크 사이즈: {list(len(text.page_context) for text in document_list)}')

1426


In [3]:
# 임베딩
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
# from langchain_chroma import Chroma
# 데이터 처음 저장할 때
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name='chroma-tax',
#     persist_directory='./chroma'
# )


In [1]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecon_api_key=os.getenv('PINECONE_API_KEY')
pc=Pinecone(api_key=pinecon_api_key)
pinecone_index=pc.Index(index_name)
# 처음 쓰고자 하는 데이터
vectorstore=PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

c:\Users\scv31\Desktop\donghwan\workspace\tax_work\taxenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# query="연봉 5천만원 거주자의 종합소득세는 얼마인가요? 결과만 주시죠"
query="연봉 5천만원 직장인 소득세는 얼마인가?"

In [6]:
retriever=vectorstore.as_retriever(search_kwargs={'k':4})
retriever.invoke(query)

[Document(id='055c5da3-c8c2-44dd-a092-37ff8bceb93a', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='b8349a60-a4a1-4d7e-aff8-663b9d5be847', metadata={'source': './data/tax.docx'}, page_content='| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |'),
 Document(id='e0b1509b-ea63-4882-b473-f57546f8df04', metadata={'source': './data/tax.docx'}, page_content='| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과

In [7]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-5')

In [8]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")     # 허깅페이스에서 제공해줌

In [ ]:
# from langchain.prompts import PromptTemplate

# prompt=PromptTemplate(
#     input_variables=['context','question'],
#     template="""
# [Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해 주세요.
# [Context]
# {context}
# [Question]
# {question}
# """
# )

In [9]:
from langchain.chains import RetrievalQA

# RetrievalQA 하나의 체인 종류임.
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt},
)

In [10]:
ai_message=qa_chain.invoke({"query":query})
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '과세표준이 5,000만원일 경우 산출소득세는 624만원입니다(= 84만원 + (5,000–1,400)만원×15%). 다만 연봉 5,000만원의 실제 소득세는 근로소득공제·인적공제·세액공제 반영 후의 과세표준에 따라 달라져 보통 이보다 낮습니다. 참고로 지방소득세(소득세의 10%)는 별도입니다.'}

In [11]:
# 한번 더 필터링
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경이 필요없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
사전:{dictionary}
질문:{{question}}
""")

dictionary_chain = prompt2 | llm | StrOutputParser()
# qa_chain을 실행하기 위한 탬플릿에 맞추기 위해서 {"query": } 형태로 가져왔음.
tex_chain = {"query": dictionary_chain} | qa_chain

In [12]:
new_question=dictionary_chain.invoke({"question":query})
new_question

'연봉 5천만원 거주자의 소득세는 얼마인가?'

In [13]:
ai_message=tex_chain.invoke({"question":query})
ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가?',
 'result': '과세표준이 연봉과 같다고 가정하면, 84만원 + (5,000만원−1,400만원)×15% = 624만원입니다. 실제로는 연봉이 곧 과세표준이 아니므로 각종 공제·비과세 적용에 따라 세액이 달라질 수 있습니다.'}